In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/AML/a part of training and testing set.zip', '/content/drive/MyDrive/AML/task2')

In [4]:
!pip install fast_ml

     |████████████████████████████████| 42 kB 527 kB/s 


In [50]:
!pip install interpret

     |████████████████████████████████| 6.6 MB 4.9 MB/s 
     |████████████████████████████████| 756 kB 52.5 MB/s 
     |████████████████████████████████| 280 kB 49.7 MB/s 
     |████████████████████████████████| 5.8 MB 56.8 MB/s 
     |████████████████████████████████| 3.6 MB 45.0 MB/s 
     |████████████████████████████████| 9.8 MB 23.6 MB/s 
     |████████████████████████████████| 793 kB 60.1 MB/s 
     |████████████████████████████████| 131 kB 71.2 MB/s 
     |████████████████████████████████| 275 kB 70.1 MB/s 
     |████████████████████████████████| 564 kB 59.0 MB/s 
     |████████████████████████████████| 251 kB 72.7 MB/s 
     |████████████████████████████████| 130 kB 75.5 MB/s 
     |████████████████████████████████| 428 kB 70.1 MB/s 
     |████████████████████████████████| 381 kB 62.5 MB/s 
     |████████████████████████████████| 81 kB 11.3 MB/s 
     |████████████████████████████████| 357 kB 57.1 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
  Created wheel fo

In [51]:
import pandas as pd
import numpy as np
import gc
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import RobustScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from fast_ml.feature_selection import get_constant_features
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import classification_report



In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/AML/task2/a part of training and testing set/UNSW_NB15_training-set.csv")
df_test = pd.read_csv("/content/drive/MyDrive/AML/task2/a part of training and testing set/UNSW_NB15_testing-set.csv")


# Data explore and preprocces

In [7]:
df_train

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.090200,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.000300,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.005100,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.660800,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.002500,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,udp,-,INT,2,0,104,0,200000.005100,...,1,2,0,0,0,2,1,0,Normal,0
82328,82329,1.106101,tcp,-,FIN,20,8,18062,354,24.410067,...,1,1,0,0,0,3,2,0,Normal,0
82329,82330,0.000000,arp,-,INT,1,0,46,0,0.000000,...,1,1,0,0,0,1,1,1,Normal,0
82330,82331,0.000000,arp,-,INT,1,0,46,0,0.000000,...,1,1,0,0,0,1,1,1,Normal,0


In [8]:
df_train.drop('id', inplace=True, axis=1)
df_test.drop('id', inplace=True, axis=1)

# Removing quasi-constant features

In [9]:
constant_features=get_constant_features(df_train,threshold=0.95,dropna=False)

In [10]:
constant_features

,Desc,Var,Value,Perc
0,Quasi Constant,is_ftp_login,0,99.176505
1,Quasi Constant,ct_ftp_cmd,0,99.174076
2,Quasi Constant,is_sm_ips_ports,0,98.887431


In [11]:
df_train.drop(['is_ftp_login','ct_ftp_cmd','is_sm_ips_ports'], inplace=True, axis=1)
df_test.drop(['is_ftp_login','ct_ftp_cmd','is_sm_ips_ports'], inplace=True, axis=1)

In [12]:
categorical = df_train.select_dtypes(include=['bool','object','datetime64']).columns.tolist()

In [13]:
categorical

['proto', 'service', 'state', 'attack_cat']

so we have 3 categorical features 

In [14]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
encoder.fit(df_train[categorical])

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [15]:
df_train[categorical]=encoder.transform(df_train[categorical])

In [16]:
df_test[categorical]=encoder.transform(df_test[categorical])

In [17]:
categorical = df_train.select_dtypes(include=['bool','object','datetime64']).columns.tolist()

In [18]:
categorical

[]

In [19]:
y_train=df_train['attack_cat']
y_test=df_test['attack_cat']

In [20]:
df_train.drop('attack_cat', inplace=True, axis=1)
df_test.drop('attack_cat', inplace=True, axis=1)

In [21]:
gc.collect()

405

In [22]:
len(df_train.dropna()) == len(df_train)

True

No nan value

# Scale the data

In [23]:
scaler = RobustScaler()

In [24]:
X_train=scaler.fit_transform(df_train.to_numpy())
X_test=scaler.transform(df_test.to_numpy())
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [25]:
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)

y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_set=TensorDataset(X_train, y_train)
test_set=TensorDataset(X_test, y_test)
batch=128
train_loader = DataLoader(train_set, batch_size=batch, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch, shuffle=True)

In [26]:
X_train.shape

torch.Size([82332, 40])

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [28]:
class Generator(nn.Module):
    
  def __init__(self, ):
    super(Generator, self).__init__()
    
    self.generator = nn.Sequential(
      nn.Linear(100, 128),#embd layer
      nn.ReLU(True),
      nn.Linear(128, 256),
      nn.ReLU(True),
      nn.Linear(256, 512),
      nn.ReLU(True),
      nn.Linear(512, 40)
    )
    
    self.generator.apply(self.__init_weights)

  def forward(self, z, y):
    y = F.one_hot(y, num_classes=10)
    
    z = torch.cat((z, y), 1)
    x = self.generator(z)
    
    return x
  
  def __init_weights(self,m):
    #Init the weights (optional)
    if type(m) == nn.Linear:
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)
      

In [29]:
class Discriminator(nn.Module):

  def __init__(self, h=128):
    super(Discriminator, self).__init__()
    
    self.discriminator = nn.Sequential(
      nn.Linear(50, 512),#embd layer
      nn.Linear(512, 256),
      nn.LeakyReLU(inplace=True),
      nn.Linear(256, 128),
      nn.LeakyReLU(inplace=True),
      nn.Linear(128, 1),
      nn.Sigmoid()
    )
    
    self.discriminator.apply(self.__init_weights)

  def forward(self, x, y):
    y = F.one_hot(y, num_classes=10)
    
    x = torch.cat((x, y), 1)
    x = self.discriminator(x)
    
    return x
  
  def __init_weights(self,m):
    #Init the weights (optional)
    if type(m) == nn.Linear:
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

In [30]:
def D_loss(out):
  loss=nn.BCELoss()
  return loss(out.squeeze(),torch.FloatTensor(out.size(0)).uniform_(0.9, 1).to(device))

In [31]:
def G_loss(out):
  loss=nn.BCELoss()
  return loss(out.squeeze(),torch.FloatTensor(out.size(0)).uniform_(0, 0.1).to(device))

In [32]:
learning_rate = 5e-4

G = Generator().to(device)
D = Discriminator().to(device)

g_opt = torch.optim.SGD(G.parameters(), lr=learning_rate)
d_opt = torch.optim.SGD(D.parameters(), lr=learning_rate)

In [35]:


for epoch in range(20):

    for batch_i, (X, y) in enumerate(train_loader):
        batch_size = X.size(0)
        z = np.random.uniform(-1, 1, size=(batch_size, 90))
        z = torch.from_numpy(z).float().to(device)
        
        d_opt.zero_grad()
        
        X = X.to(device)
        y = y.to(device)
        
        d_real = D(X, y)
        d_loss = D_loss(d_real)
        
        g_fake = G(z, y)
        
        d_fake = D(g_fake, y)
        d_loss = d_loss + G_loss(d_fake)
        
        d_loss.backward()
        d_opt.step()

        g_opt.zero_grad()
        
        g_fake = G(z, y)
        

        g_loss = D_loss(D(g_fake, y))
        
        g_loss.backward()
        g_opt.step()
            
    print(epoch + 1, d_loss.item(), g_loss.item())
    

1 1.0877330303192139 1.759049892425537
2 1.0564074516296387 1.7447971105575562
3 1.6525793075561523 1.7561821937561035
4 1.1890037059783936 1.6169116497039795
5 0.8162331581115723 1.9150240421295166
6 2.2430925369262695 1.7592624425888062
7 1.313297986984253 2.0761094093322754
8 1.4861171245574951 1.698443055152893
9 1.1643136739730835 1.566138505935669
10 1.2268688678741455 1.8431024551391602
11 0.8901405334472656 1.9780957698822021
12 0.7582005262374878 2.014357566833496
13 1.0799236297607422 1.9394885301589966
14 1.117194414138794 1.70563542842865
15 0.8689202070236206 2.1902542114257812
16 1.9658085107803345 1.8180744647979736
17 0.7438187599182129 1.9000258445739746
18 0.8706247806549072 1.7065212726593018
19 1.2435672283172607 1.9928398132324219
20 0.6581801772117615 1.8940818309783936


In [79]:
clf=RandomForestClassifier()

In [57]:
clf.fit(X_train.numpy(),y_train.numpy())

RandomForestClassifier()

In [54]:
y_pred=clf.predict(X_test.numpy())

In [55]:
print(classification_report(y_pred,y_test.numpy()))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.05      0.97      0.10        92
           2       0.56      0.34      0.43     19938
           3       0.65      0.73      0.69     29919
           4       0.86      0.90      0.88     17437
           5       0.98      0.94      0.96     41827
           6       1.00      1.00      1.00     56006
           7       0.74      0.92      0.82      8406
           8       0.65      0.44      0.52      1685
           9       0.17      0.71      0.27        31

    accuracy                           0.85    175341
   macro avg       0.57      0.70      0.57    175341
weighted avg       0.86      0.85      0.85    175341



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
clf = ExplainableBoostingClassifier(max_bins=3, n_jobs=-1, interactions=0, max_leaves=25)


In [59]:
clf.fit(X_train.numpy(),y_train.numpy())

/usr/local/lib/python3.7/dist-packages/interpret/glassbox/ebm/ebm.py:922: UserWarning: Multiclass is still experimental. Subject to change per release.
  warn("Multiclass is still experimental. Subject to change per release.")
/usr/local/lib/python3.7/dist-packages/interpret/glassbox/ebm/ebm.py:925: UserWarning: Detected multiclass problem: forcing interactions to 0
  warn("Detected multiclass problem: forcing interactions to 0")


ExplainableBoostingClassifier(feature_names=['feature_0001', 'feature_0002',
                                             'feature_0003', 'feature_0004',
                                             'feature_0005', 'feature_0006',
                                             'feature_0007', 'feature_0008',
                                             'feature_0009', 'feature_0010',
                                             'feature_0011', 'feature_0012',
                                             'feature_0013', 'feature_0014',
                                             'feature_0015', 'feature_0016',
                                             'feature_0017', 'feature_0018',
                                             'feature_0019', 'feature_0020',
                                             'feat...
                                             'continuous', 'continuous',
                                             'continuous', 'continuous',
                              

In [60]:
y_pred=clf.predict(X_test.numpy())

In [61]:
print(classification_report(y_pred,y_test.numpy()))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.01      0.71      0.01        17
           2       0.49      0.33      0.39     18473
           3       0.68      0.69      0.68     32758
           4       0.84      0.85      0.85     18006
           5       0.98      0.99      0.98     39773
           6       1.00      1.00      1.00     56000
           7       0.75      0.85      0.80      9172
           8       0.56      0.57      0.56      1118
           9       0.11      0.58      0.18        24

    accuracy                           0.84    175341
   macro avg       0.54      0.66      0.55    175341
weighted avg       0.85      0.84      0.84    175341



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
class NN(nn.Module):
    
  def __init__(self):
    super(NN, self).__init__()
    
    self.clf = nn.Sequential(
      nn.Linear(40, 30),
      nn.ReLU(True),
      nn.Linear(30, 10)
    )
    
    self.clf.apply(self.__init_weights)

  def forward(self, x):
    out = self.clf(x)
    
    return out
  
  def __init_weights(self,m):
    #Init the weights (optional)
    if type(m) == nn.Linear:
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

In [65]:
def trainer(model, data):

    optimizer = torch.optim.SGD(model.parameters(), lr=5e-4)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    
    for epoch in range(20):
        for X, y in data:
            X = X.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            
            loss.backward()
            optimizer.step()
        
    return model

In [66]:
clf=NN().to(device)

In [ ]:
trainer(clf,train_loader)
clf.eval()


In [68]:
out=clf(X_test.to(device))
y_pred = torch.max(out.data, 1)[1].cpu().detach().numpy()

In [69]:
print(classification_report(y_pred,y_test.numpy()))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.00      0.00      0.00         0
           2       0.09      0.33      0.14      3366
           3       0.47      0.63      0.54     25192
           4       0.14      0.53      0.22      4818
           5       0.98      0.77      0.86     51049
           6       0.97      0.60      0.74     90769
           7       0.00      0.05      0.00       137
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         0

    accuracy                           0.64    175341
   macro avg       0.26      0.29      0.25    175341
weighted avg       0.86      0.64      0.72    175341



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
label,occ=np.unique(y_train,return_counts=True)

In [71]:
max_occ_index=label[occ == np.amax(occ)][0]

In [72]:
label,occ

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([  677,   583,  4089, 11132,  6062, 18871, 37000,  3496,   378,
           44]))

In [73]:
gen_data={}
for item in label:
  size = occ[max_occ_index] - occ[item]
  y = torch.ones((size), dtype=torch.int64).to(device) * item
  z = np.random.uniform(-1, 1, size=(size, 90))
  z = torch.from_numpy(z).float().to(device)

  gen_data[item]= G(z, y).cpu().detach().numpy()

In [74]:
gen_data[0]

array([[ 0.8022051 , -0.53289634,  0.6125934 , ..., -0.6416722 ,
        -0.24443264, -0.42061332],
       [ 0.26957148, -1.0237646 ,  0.64211535, ..., -0.693733  ,
        -0.20636961, -0.4649241 ],
       [ 0.64323866, -0.79318404,  0.6918823 , ..., -0.6443804 ,
        -0.39794752, -0.36559632],
       ...,
       [ 0.30449992, -0.8393423 ,  1.2148141 , ..., -0.6889696 ,
        -0.23108073, -0.47350612],
       [ 0.08554848, -0.63500667,  0.9741882 , ..., -0.36863837,
        -0.0339376 , -0.40630844],
       [ 0.38178557, -0.69739074,  0.8900523 , ..., -0.88767433,
        -0.32681608, -0.24115892]], dtype=float32)

In [75]:
new_X = []
new_y = []
for label, data in gen_data.items():
    size = len(data)
    new_y.append(np.ones(size) * label)
    new_X.append(data)
new_X.append(X_train.numpy())
new_y.append(y_train.numpy())
new_X_train = np.concatenate(new_X)
new_y_train = np.concatenate(new_y)

In [76]:
si = np.random.permutation(len(new_X_train))

new_X_train= new_X_train[si]
new_y_train = new_y_train[si]

new_X_train_t = torch.tensor(new_X_train)
new_y_train_t = torch.tensor(new_y_train, dtype=torch.int64)

new_train_loader = DataLoader(TensorDataset(new_X_train_t, new_y_train_t), batch_size=128, shuffle=True)

In [82]:
clf=RandomForestClassifier()

In [83]:
clf.fit(new_X_train,new_y_train)

RandomForestClassifier()

In [84]:
y_pred=clf.predict(X_test.numpy())

In [85]:
print(classification_report(y_pred,y_test.numpy()))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       0.05      0.98      0.10        93
         2.0       0.56      0.34      0.43     20159
         3.0       0.65      0.73      0.69     29563
         4.0       0.86      0.89      0.88     17503
         5.0       0.98      0.94      0.96     41968
         6.0       1.00      1.00      1.00     56000
         7.0       0.74      0.92      0.82      8430
         8.0       0.64      0.46      0.53      1596
         9.0       0.16      0.72      0.26        29

    accuracy                           0.84    175341
   macro avg       0.57      0.70      0.57    175341
weighted avg       0.86      0.84      0.85    175341



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
clf = ExplainableBoostingClassifier(max_bins=3, n_jobs=-1, interactions=0, max_leaves=25)


In [ ]:
clf.fit(new_X_train,new_y_train)

/usr/local/lib/python3.7/dist-packages/interpret/glassbox/ebm/ebm.py:922: UserWarning: Multiclass is still experimental. Subject to change per release.
  warn("Multiclass is still experimental. Subject to change per release.")


In [ ]:
y_pred=clf.predict(X_test.numpy())

In [ ]:
print(classification_report(y_pred,y_test.numpy()))

In [ ]:
clf=NN().to(device)

In [ ]:
trainer(clf,new_train_loader)
clf.eval()


In [ ]:
out=clf(X_test.to(device))
y_pred = torch.max(out.data, 1)[1].cpu().detach().numpy()

In [ ]:
print(classification_report(y_pred,y_test.numpy()))